In [4]:
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy import stats

def mse(predictions, targets):
    count = 0
    k = []
    for x in predictions:
        temp = (((predictions[count] - targets[count]) ** 2)/2)
        count+=1
        k.append(temp)
    return k


# df = pd.read_csv('drop_out.csv')
df = pd.read_csv('fill.csv')


# print(X.columns)
Vet = df['PCV_target']
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC', 'PLT\n______', 'HGB', 'RBC', 'MCV', 'MCHC', 'MCH',
#         'SEGS', 'LYMPH', 'MONO','RDW']]
X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume']]
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'MCV', 'MCHC', 'MCH']]
y = df['PCV_afterdonation']
column_name = X.columns

# # convert to numeric
# X = X.apply(pd.to_numeric, errors='coerce')
# y = y.apply(pd.to_numeric, errors='coerce')
# Vet = Vet.apply(pd.to_numeric, errors='coerce')



# # fill with mean
# X = X.fillna(X.mean())
# y = y.fillna(y.mean())
# Vet = Vet.fillna(Vet.mean())


X = np.array(X)
y = np.array(y)
Vet = np.array(Vet)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


kf = KFold(n_splits=10)
kf.get_n_splits(X)

MSE=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    temp_MSE = (metrics.mean_squared_error(y_test, predictions))
    MSE.append(temp_MSE)
#     print(temp_MSE)


    
print ('Average MSE 10 fold cross validation : %f'%np.mean(MSE))


MSE=[]
train_MSE=[]
predict_for_p=[]
# print("leave one out")

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

# print(loo)
coef_list = []
for train_index, test_index in loo.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#    print(X_train, X_test, y_train, y_test)

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    
#     for train accuracy
    xxx = lm.predict(X_train)
    temp_train_MSE = (metrics.mean_squared_error(y_train, xxx))
    train_MSE.append(temp_train_MSE)
#     asdfafdafd


    predictions = lm.predict(X_test)
    temp_MSE = (metrics.mean_squared_error(y_test, predictions))
    MSE.append(temp_MSE)
    predict_for_p.append(predictions)
#     print(lm.coef_)
    coef_list.append(lm.coef_)
#     print(temp_MSE)
print ('Average MSE Leave one out : %f ± %f'%(np.mean(MSE), np.std(MSE)))
print ('average train MSE  : %f ± %f'%(np.mean(train_MSE), np.std(train_MSE)))

print("********")
mse_vet = mse(Vet,y)
# print(mse_vet)
# print(stats.ttest_rel(MSE,mse_vet).pvalue)
p_value = stats.ttest_rel(MSE,mse_vet).pvalue
print('P value is %f'%p_value)


print ('MSE Vet : %f'%((metrics.mean_squared_error(Vet, y))))
print ('Average Coef')
coef_list = np.matrix(coef_list)
coef_list = np.absolute(coef_list)
# print(column_name)
x = coef_list.mean(0)
# print (x)
array_x = np.array(x)
# print(array_x)

coeff_df = pd.DataFrame(array_x[0],column_name,columns=['Coef'])
coeff_df = coeff_df.sort_values(by=['Coef'], ascending=False)
print(coeff_df)

Average MSE 10 fold cross validation : 35.037286
Average MSE Leave one out : 35.332388 ± 70.577842
average train MSE  : 28.884204 ± 0.885170
********
P value is 0.148427
MSE Vet : 41.892226
Average Coef
                Coef
PCV         6.469751
Weight      3.899351
PCV\ndonor  1.174907
Volume      0.714401
